In [1]:
import os, pdb, sys, json, math
from lumina.genetic.metrics.ts_pnl import calculate_ful_ts_ret
from lumina.genetic.signal.method import *
from lumina.genetic.strategy.method import *
from lumina.genetic.process import *
from ultron.factor.genetic.geneticist.operators import *
import ultron.factor.empyrical as empyrical

from dotenv import load_dotenv
load_dotenv()

from kdutils.common import *
from kdutils.macro2 import *

In [2]:
from ultron.factor.genetic.geneticist.operators import *

In [3]:
eval("RELU(SIGLOG10ABS(WMA(10, 'close') - MMinDiff(5, 'volume')) + WMA(20, 'ixy013_5_10_1'))")

In [2]:
name = 'train'
method = 'aicso0'
task_id = '113001'
instruments = "rbb"

In [3]:
total_data = fetch_temp_data(
            method=method,
            task_id=task_id,
            instruments=instruments,
            datasets=name if isinstance(name, list) else [name])

total_data = total_data.drop_duplicates(
    subset=['trade_time', 'code']).reset_index(drop=True)
total_data.head()

./records/aicso0/rbb/basic/train_data.feather


,trade_time,code,fz002_10_15_0,fz002_10_15_1,fz002_5_10_0,fz002_5_10_1,gd002_10_15_0,gd002_10_15_1,gd002_5_10_0,gd002_5_10_1,...,tf022_5_10_0,tf022_5_10_1,close,high,low,open,value,volume,openint,vwap
0,2020-01-02 09:24:00,RB,0.000017,0.000046,0.000016,0.000033,0.523321,0.491670,0.490432,0.493088,...,3863.813874,3864.368054,3867.117696,3867.117696,3864.962112,3866.039904,101900660.0,2841.0,1477768.0,3586.788455
1,2020-01-02 09:25:00,RB,0.000008,0.000042,0.000014,0.000028,0.529929,0.498302,0.513677,0.514292,...,3864.102655,3864.852516,3866.039904,3868.195488,3864.962112,3867.117696,148793300.0,4147.0,1478987.0,3587.974439
2,2020-01-02 09:26:00,RB,0.000007,0.000037,0.000010,0.000020,0.533049,0.508534,0.558912,0.543030,...,3864.316665,3864.794220,3863.884320,3866.039904,3863.884320,3866.039904,70353230.0,1962.0,1479086.0,3585.791539
3,2020-01-02 09:27:00,RB,0.000007,0.000032,0.000005,0.000011,0.536877,0.526550,0.599309,0.589248,...,3863.817898,3863.609003,3862.806528,3863.884320,3860.650944,3863.884320,148451470.0,4143.0,1479142.0,3583.187787
4,2020-01-02 09:28:00,RB,0.000007,0.000028,0.000002,0.000007,0.529006,0.543816,0.634817,0.626719,...,3863.318107,3863.518597,3863.884320,3863.884320,3862.806528,3863.884320,40038130.0,1117.0,1479121.0,3584.434199


In [4]:
def calcute_strategy(total_data, column, name, method, instruments, task_id):
        backup_cycle = 1
        ## 计算
        total_data1 = total_data.set_index(['trade_time'])
        #total_data2 = total_data.set_index(['trade_time', 'code']).unstack()

        expression = column['formual']
        signal_method = column['signal_method']
        strategy_method = column['strategy_method']
        signal_params = column['signal_params']
        signal_params = {
            key: value
            for key, value in signal_params.items() if value is not None
        }
        strategy_params = column['strategy_params']
        strategy_params = {
            key: value
            for key, value in strategy_params.items() if value is not None
        }

        ### 保持和挖掘一致， 要做inf值处理，要做极小值处理
        factor_data = calc_factor(expression=expression,
                                  total_data=total_data1,
                                  indexs=[],
                                  key='code')
        factor_data = factor_data.replace([np.inf, -np.inf], np.nan)
        factor_data['transformed'] = np.where(
            np.abs(factor_data.transformed.values) > 0.000001,
            factor_data.transformed.values, np.nan)
        factor_data = factor_data.loc[factor_data.index.unique()
                                      [backup_cycle:]]

        factors_data1 = factor_data.reset_index().set_index(
            ['trade_time', 'code'])

        cycle_total_data = total_data.copy()
        cycle_total_data = cycle_total_data.loc[
            cycle_total_data.index.unique()[backup_cycle:]]

        total_data1 = cycle_total_data.reset_index().set_index(
            ['trade_time', 'code']).unstack()

        pos_data = eval(signal_method)(factor_data=factors_data1,
                                       **signal_params)
        pos_data1 = eval(strategy_method)(signal=pos_data,
                                          total_data=total_data1,
                                          **strategy_params)

In [7]:
columns = {
    'formual': "MDIFF(16,'oi032_5_5_10_0')",
    "signal_method": "mean_signal",
    "signal_params": {
        'roll_num': 50,
        'threshold': 0.2
    },
    "strategy_method":"trailing_atr_strategy",
    "strategy_params":{"atr_multiplier":9.0,
    "atr_multiplier":30,"max_volume":1}

}
calcute_strategy(
    total_data=total_data,
    column=columns,
                 name=name,
                 method=method,
                 instruments=instruments,
                 task_id=task_id)

TypeError: trailing_atr_strategy() missing 1 required positional argument: 'atr_period'